# Top10 most liked and retweeted tweets (year and month)


Libraries

In [64]:
#to clean data
import pandas as pd
import nltk
import re

from collections import Counter,OrderedDict
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist


Dataframe

In [65]:
df =  pd.read_csv('tweet_all2.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')

print(len(df))
df.head(5)

1170852


,id,text,created_at,author_id,lang,in_reply_to_user_id
0,1000016383915253765,"@CleberEsteves09, bom dia. A Imigrantes, neste...",2018-05-25 14:10:59,138075168,pt,96735968
1,1000016491390099456,Oposição cabo-verdiana entrega projeto para re...,2018-05-25 14:11:25,1690412382,pt,None
2,1000016689931653123,"""Cheguei a São Paulo como refugiado. Tinha 16 ...",2018-05-25 14:12:12,2149024418,pt,None
3,1000016951853318145,"""Cheguei a São Paulo como refugiado. Tinha 16 ...",2018-05-25 14:13:15,26411144,pt,None
4,1000017172918358016,@Grettux_MG @SenhooraDepp @KaySsicaBaby No tt ...,2018-05-25 14:14:08,818123243924160513,pt,2493509342


'lang' filter (still not sure if to use it or not)

In [23]:
# Filter rows where the language is 'pt'
df_pt = df[df['lang'] == 'pt']

print(len(df_pt))

750071


Cleaning 

In [56]:
# Specify the list of specific words you want to filter out
specific_words = ['trânsito', 'rodovia', 'tráfego', 'ecovias','TIM', 'tim', 'Kaysar' , 'novela', 'Bandeirantes', 'operacaobetalab', 'bbb', 'transito', 'trânsito', 'orcars', 'oscar', 'grammy', 'grammys', 'kaysar', 'timbetalab', ' masterchefbr', 'enem', 'futebol', ' bundesliga', 'corinthians']

# Fill NaN values in the 'text' column with an empty string
df_pt['text'].fillna('', inplace=True)

# Create a boolean mask for rows containing specific words in the 'text' column
mask = df_pt['text'].str.contains('|'.join(specific_words), case=False)

# Apply the mask to filter out rows with specific words
df_filtered = df_pt[~mask]

# Define a regular expression pattern to match URLs
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# Remove URLs from the 'text' column using regular expressions
df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(url_pattern, '', regex=True)

print("Filtered DataFrame length:", len(df_filtered))


Filtered DataFrame length: 666895


Merging filtered df with public_metrics

In [57]:
df1 = df_filtered
df2 = pd.read_csv('public_metrics.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')

# Convert 'id' column in df1 to string
df1['id'] = df1['id'].astype(str)

# Convert 'tweet' column in df2 to string
df2['tweet'] = df2['tweet'].astype(str)

# Fill NaN values in the 'tweet' column of df2
df2['tweet'].fillna('', inplace=True)

# Perform an inner join on 'id' and 'tweet' columns to include only rows with correspondence
merged_df = df1.merge(df2, left_on='id', right_on='tweet', how='inner')

# Print the length of the new merged dataframe
print("Filtered DataFrame length:", len(merged_df))

Filtered DataFrame length: 623361


Parsing data by year

In [58]:
# Define the parse_date function
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT  

# Apply the parse_date function to 'created_at' column
merged_df['created_at'] = merged_df['created_at'].apply(parse_date)

# Drop rows with missing dates
merged_df = merged_df.dropna(subset=['created_at'])

# Extract the 'date' and 'time' components
merged_df['date'] = merged_df['created_at'].dt.date
merged_df['time'] = merged_df['created_at'].dt.time

# Filter rows for the years 2015 to 2019
filtered_df = merged_df[(merged_df['created_at'] >= pd.Timestamp('2015-01-01')) & (merged_df['created_at'] <= pd.Timestamp('2019-12-31'))]

# Group the data by year and count the occurrences
year_frequency = filtered_df.groupby(filtered_df['created_at'].dt.year)['date'].count()

print(year_frequency)

created_at
2015    132272
2016    148870
2017    135837
2018     81890
2019    124228
Name: date, dtype: int64


Parsing data by month

In [63]:
# Filter rows per year
filtered_df = merged_df[(merged_df['created_at'] >= pd.Timestamp('2015-01-01')) & (merged_df['created_at'] <= pd.Timestamp('2019-12-31'))]

# Group the data by year and month and count the occurrences
month_frequency = filtered_df.groupby([filtered_df['created_at'].dt.year, filtered_df['created_at'].dt.month])['date'].count()

print(month_frequency)

created_at  created_at
2015        2              8497
            3              7752
            4             10690
            5             13144
            6             12736
            7             11174
            8             14122
            9             14618
            10            13458
            11            13563
            12            12518
2016        1             12814
            2             12041
            3             12777
            4             12486
            5             12285
            6             12652
            7             12433
            8             12094
            9             12817
            10            12811
            11            11900
            12            11760
2017        1             11747
            2             12189
            3             12206
            4             11343
            5             12777
            6             11887
            7             10393
            8    

# LIKES

In [69]:
# Filter rows per year 
filtered_df = merged_df[(merged_df['created_at'] >= pd.Timestamp('2015-01-01')) & (merged_df['created_at'] <= pd.Timestamp('2015-12-31'))]

# Filter rows per month 
monthyear_like = filtered_df[(filtered_df['created_at'].dt.year == 2015) & (filtered_df['created_at'].dt.month == 4)]

# Get the top 10 rows from the column 'text' along with 'like_count'
top_10_likes = monthyear_like.nlargest(10, 'like_count')[['like_count', 'text']]

# Print the top 10 rows with 'like_count' and text
for index, row in top_10_likes.iterrows():
    print("Row", index)
    print("Likes:", row['like_count'])
    print("Text:", row['text'])
    print("=" * 30)  

Row 186670
Likes: 167
Text: Jolie: 'É repugnante ver refugiados que se afogam no limiar do continente mais rico do mundo'. http://t.co/PKQ8qWE4t7 http://t.co/FeAifm79Ti
Row 181338
Likes: 87
Text: Imigrantes venezuelanos fazem panelaço perto de hotel onde Maduro está. #CumbrePanama http://t.co/Mng9kcJeQB http://t.co/lJ8AkYd4A2
Row 188384
Likes: 74
Text: Obama premia professora de alunos refugiados e pede melhores salários aos mestres http://t.co/AfWR5LI4Qw #G1 http://t.co/xAo662wmP9
Row 184366
Likes: 72
Text: Filme conta como é ser dançarino em um país onde dançar é pecado. http://t.co/XYl35Jp3Xt http://t.co/EcNc4I2gDN
Row 179589
Likes: 67
Text: 2 mil pessoas são retiradas de campo de refugiados na Síria com avanço do Estado Islâmico http://t.co/2w1ji8BEG4 http://t.co/64539bJTYr
Row 184298
Likes: 66
Text: O #Fantastico foi até a ilha de Lampedusa, na Itália, para mostrar de perto como é a chegada dos refugiados
Row 188056
Likes: 59
Text: Nepal: Índia dará visto gratuito para brasileiros

# RETWEETS (weird result)


In [71]:
# Filter rows per year 
filtered_df = merged_df[(merged_df['created_at'] >= pd.Timestamp('2015-01-01')) & (merged_df['created_at'] <= pd.Timestamp('2015-12-31'))]

# Filter rows per month 
monthyearfilter_rt = filtered_df[(filtered_df['created_at'].dt.year == 2015) & (filtered_df['created_at'].dt.month == 3)]

# Get the top 10 rows from the column 'text' along with 'like_count'
top_10_retweets = monthyearfilter_rt.nlargest(10, 'retweet_count')[['retweet_count', 'text']]

# Print the top 10 rows with 'like_count' and text
for index, row in top_10_retweets.iterrows():
    print("Row", index)
    print("Retweets:", row['retweet_count'])
    print("Text:", row['text'])
    print("=" * 30)  

Row 173585
Retweets: 163
Text: RT @AlfioKrancic: Isis migrante... http://t.co/1kQAXBGrFY
Row 173724
Retweets: 163
Text: RT @AlfioKrancic: Isis migrante... http://t.co/1kQAXBGrFY
Row 176797
Retweets: 163
Text: RT @AlfioKrancic: Isis migrante... http://t.co/1kQAXBGrFY
Row 176802
Retweets: 163
Text: RT @AlfioKrancic: Isis migrante... http://t.co/1kQAXBGrFY
Row 175174
Retweets: 104
Text: Além de convocar Ideli, vou orientar cubanos a pedirem no Comitê Nacional para os Refugiados (Conare) refúgio para eles e suas famílias.
Row 175175
Retweets: 104
Text: RT @SenadorCaiado: Além de convocar Ideli, vou orientar cubanos a pedirem no Comitê Nacional para os Refugiados (Conare) refúgio para eles …
Row 175176
Retweets: 104
Text: RT @SenadorCaiado: Além de convocar Ideli, vou orientar cubanos a pedirem no Comitê Nacional para os Refugiados (Conare) refúgio para eles …
Row 175177
Retweets: 104
Text: RT @SenadorCaiado: Além de convocar Ideli, vou orientar cubanos a pedirem no Comitê Nacional para os 